To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://github.com/unslothai/unsloth?tab=readme-ov-file#-installation-instructions).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save) (eg for Llama.cpp).

**[NEW] Gemma2-9b is trained on 8 trillion tokens! Gemma2-27b is 13 trillion!**

In [1]:
# %%capture
# !pip install unsloth
# # Also get the latest nightly Unsloth!
# !pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# # Install Flash Attention 2 for softcapping support
# import torch
# if torch.cuda.get_device_capability()[0] >= 8:
#     !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

* We support Llama, Mistral, Phi-3, Gemma, Yi, DeepSeek, Qwen, TinyLlama, Vicuna, Open Hermes etc
* We support 16bit LoRA or 4bit QLoRA. Both 2x faster.
* `max_seq_length` can be set to anything, since we do automatic RoPE Scaling via [kaiokendev's](https://kaiokendev.github.io/til) method.
* With [PR 26037](https://github.com/huggingface/transformers/pull/26037), we support downloading 4bit models **4x faster**! [Our repo](https://huggingface.co/unsloth) has Llama, Mistral 4bit models.
* [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)

In [2]:
from unsloth import FastLanguageModel
import torch
import os, dotenv
dotenv.load_dotenv()
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
hf_token = os.getenv('HUGGINGFACEHUB_API_TOKEN')

alpaca_prompt = """
### Instruction:
{}

### Input:
{}

### Response:
{}"""

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [3]:
# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "google/gemma-2-27b-it",  # google/gemma-2-27b-it | unsloth/gemma-2-27b-bnb-4bit
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.9.post4: Fast Gemma2 patching. Transformers = 4.45.1.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.551 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.9.post4 patched 46 layers with 46 QKV layers, 46 O layers and 46 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [5]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
# dataset = load_dataset("flytech/python-codes-25k", split = "train")
data_files = {
    'train': './dataset/generated_dataset_train-0.5.csv',
    'test': './dataset/generated_dataset_test-0.5.csv'
}
dataset = load_dataset('csv', data_files=data_files)
dataset = dataset.map(formatting_prompts_func, batched = True)

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'],
    eval_dataset = dataset['test'],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        num_train_epochs=1,
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # max_steps = 20,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [9]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4090. Max memory = 23.551 GB.
15.502 GB of memory reserved.


In [10]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 8,981 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 4
\        /    Total batch size = 32 | Total steps = 280
 "-____-"     Number of trainable parameters = 114,180,096


  0%|          | 0/280 [00:00<?, ?it/s]

{'loss': 1.8757, 'grad_norm': 1.094877004623413, 'learning_rate': 4e-05, 'epoch': 0.0}
{'loss': 1.9727, 'grad_norm': 1.142242670059204, 'learning_rate': 8e-05, 'epoch': 0.01}
{'loss': 1.951, 'grad_norm': 1.1390976905822754, 'learning_rate': 0.00012, 'epoch': 0.01}
{'loss': 1.794, 'grad_norm': 1.1784157752990723, 'learning_rate': 0.00016, 'epoch': 0.01}
{'loss': 1.5456, 'grad_norm': 1.1980695724487305, 'learning_rate': 0.0002, 'epoch': 0.02}
{'loss': 1.2425, 'grad_norm': 1.061708927154541, 'learning_rate': 0.00019927272727272727, 'epoch': 0.02}
{'loss': 1.0503, 'grad_norm': 1.0565959215164185, 'learning_rate': 0.00019854545454545456, 'epoch': 0.02}
{'loss': 0.9393, 'grad_norm': 1.2532984018325806, 'learning_rate': 0.00019781818181818184, 'epoch': 0.03}
{'loss': 0.8927, 'grad_norm': 1.2818683385849, 'learning_rate': 0.0001970909090909091, 'epoch': 0.03}
{'loss': 0.7789, 'grad_norm': 0.9967097043991089, 'learning_rate': 0.00019636363636363636, 'epoch': 0.04}
{'loss': 0.7651, 'grad_norm': 

In [11]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory/max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

4493.4634 seconds used for training.
74.89 minutes used for training.
Peak reserved memory = 22.352 GB.
Peak reserved memory for training = 6.85 GB.
Peak reserved memory % of max memory = 94.909 %.
Peak reserved memory for training % of max memory = 29.086 %.


In [12]:
from tqdm import tqdm
results = []
batch_size = 1
FastLanguageModel.for_inference(model)
for i in tqdm(range(0, len(dataset['test']), batch_size)):
    instructions = dataset['test']['instruction'][i:i + batch_size]
    inputs = dataset['test']['input'][i:i + batch_size]

    inputs = tokenizer(
        [
            alpaca_prompt.format(
                instructions,
                inputs, 
                "", 
            ) for instructions, inputs in zip(instructions, inputs)
        ], return_tensors="pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens=256, use_cache=True)
    results.extend(tokenizer.batch_decode(outputs))

100%|██████████| 184/184 [18:46<00:00,  6.12s/it]


In [13]:
results = [i.split('### Response:\n')[-1].replace('<eos>', '') for i in results]
results

['Как вы думаете, что может произойти, если сервер не отвечает на ваш запрос? Обратите внимание на то, как вы обрабатываете ответ от сервера. Есть ли у вас возможность проверить, что именно возвращает сервер в случае ошибки? Подумайте, как можно улучшить обработку ошибок в вашем коде, чтобы получить больше информации о том, что пошло не так.',
 'Как вы думаете, сколько времени занимает выполнение вашей функции? Может быть, стоит задуматься о том, как это может повлиять на выполнение вашего кода? Есть ли у вас идеи, как можно изменить подход к выполнению этой функции, чтобы избежать задержек? Попробуйте рассмотреть, как можно оптимизировать время выполнения или использовать другие методы для достижения желаемого результата.',
 'Как вы думаете, что произойдёт, если вы попытаетесь сложить два разных типа данных? Может быть, стоит задуматься о том, как вы обрабатываете входные значения в функции? Попробуйте рассмотреть, какие типы данных вы используете и как они могут взаимодействовать дру

In [14]:
from transformers import BertModel, BertTokenizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import torch

# Инициализация модели и токенизатора BERT
model_name = 'google-bert/bert-base-multilingual-cased'  # Вы можете выбрать другую предобученную модель
bert_tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = BertModel.from_pretrained(model_name)

def get_embedding(text):
    """
    Получает эмбеддинг для заданного текста с использованием BERT.
    """
    # Токенизация текста
    inputs = bert_tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    
    with torch.no_grad():
        outputs = bert_model(**inputs)
    
    # Получение эмбеддинга [CLS] токена
    cls_embedding = outputs.last_hidden_state[:, 0, :]  # Размерность (1, скрытая размерность)
    return cls_embedding.numpy()

def compute_cosine_distance(embedding1, embedding2):
    """
    Вычисляет косинусное расстояние между двумя эмбеддингами.
    """
    # Косинусное сходство
    similarity = cosine_similarity(embedding1, embedding2)
    # Косинусное расстояние
    distance = 1 - similarity
    return distance[0][0]

In [15]:
# Получение эмбеддингов для всех комментариев
user_embeddings = [get_embedding(comment) for comment in results]
instructor_embeddings = [get_embedding(comment) for comment in dataset['test']['output']]

# Вычисление средней эмбеддингов для пользователя и преподавателя
user_mean_embedding = np.mean(user_embeddings, axis=0).reshape(1, -1)
instructor_mean_embedding = np.mean(instructor_embeddings, axis=0).reshape(1, -1)

# Вычисление косинусного расстояния
cosine_distance = compute_cosine_distance(user_mean_embedding, instructor_mean_embedding)
print(f"Косинусное расстояние: {cosine_distance:.6f}")

Косинусное расстояние: 0.000870


In [16]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        dataset['test']['instruction'][0], # instruction
        dataset['test']['input'][0], # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
tokenizer.batch_decode(outputs)[0]

'<bos>\n### Instruction:\nВы учитель и вы должны на русском языке давать наводящие подсказки без прямых исправлений кода, поддерживая формальный и дружелюбный стиль общения\n\n### Input:\n```python\nimport requests\n\ndef fetch_data(url):\n    response = requests.get(url)\n    if response.status_code == 200:\n        return response.json()\n    else:\n        return None\n\nurl = "https://api.example.com/data"\ndata = fetch_data(url)\nprint(data)\n```\n\n### Response:\nКак вы думаете, что может произойти, если сервер не отвечает на ваш запрос? Обратите внимание на то, как вы обрабатываете ответ от сервера. Есть ли у вас возможность проверить, что именно возвращает сервер в случае ошибки? Подумайте, как можно улучшить обработку ошибок в вашем коде, чтобы получить больше информации о том, что пошло не так.<eos>'

<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [23]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Почини мою программу", # instruction
        "lambda x, y: x/(x+y); f(1, 9)", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['[CLS] # # # Instruction : Почини мою программу # # # Input : lambda x, y : x / ( x + y ) ; f ( 1, 9 ) # # # Response : [SEP] # # [unused1]']

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [18]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        dataset['test']['instruction'][0], # instruction
        dataset['test']['input'][0], # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 256)

<bos>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Вы учитель и вы должны на русском языке давать наводящие подсказки без прямых исправлений кода, поддерживая формальный и дружелюбный стиль общения

### Input:
```python
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n - 1)

print(factorial(1000))
```

### Response:
Подумайте, что происходит, когда вы вызываете функцию с большим значением. Как это может повлиять на работу программы? Обратите внимание на то, как функция вызывается сама из себя. Что может произойти, если это повторение будет продолжаться слишком долго?<eos>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [12]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
model.push_to_hub("kekwak/gemma-2-27b", token = hf_token) # Online saving
tokenizer.push_to_hub("kekwak/gemma-2-27b", token = hf_token) # Online saving

adapter_model.safetensors: 100%|██████████| 457M/457M [00:18<00:00, 24.8MB/s] 


Saved model to https://huggingface.co/kekwak/gemma-2-27b


tokenizer.json: 100%|██████████| 34.4M/34.4M [00:01<00:00, 28.3MB/s]


Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [2]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

==((====))==  Unsloth 2024.9.post4: Fast Gemma2 patching. Transformers = 4.45.1.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.551 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.91s/it]
Unsloth 2024.9.post4 patched 46 layers with 46 QKV layers, 46 O layers and 46 MLP layers.


['<bos>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat is a famous tall tower in Paris?\n\n### Input:\n\n\n### Response:\nThe Eiffel Tower<eos>']

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [3]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("kekwak/gemma-2-27b", tokenizer, save_method = "merged_16bit", token = hf_token)

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit_forced", safe_serialization = False)  # !!! merged_4bit !!!
if True: model.push_to_hub_merged("kekwak/gemma-2-27b", tokenizer, save_method = "merged_4bit_forced", safe_serialization = False, token = hf_token)

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("kekwak/gemma-2-27b", tokenizer, save_method = "lora", token = hf_token)

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [3]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("kekwak/gemma-2-27b", tokenizer, token = hf_token)

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("kekwak/gemma-2-27b", tokenizer, quantization_method = "f16", token = hf_token)

# Save to q4_k_m GGUF
if True: model.save_pretrained_gguf("gemma-2-27b", tokenizer, quantization_method = "q5_k_m")
# if True: model.push_to_hub_gguf("kekwak/gemma-2-27b", tokenizer, quantization_method = "q5_k_m", first_conversion="q8_0", token = hf_token)

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 16.92 out of 31.25 RAM for saving.


100%|██████████| 46/46 [00:42<00:00,  1.08it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


Unsloth: Converting gemma2 model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q5_k_m'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at gemma-2-27b into bf16 GGUF format.
The output location will be ./gemma-2-27b/unsloth.BF16.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: gemma-2-27b
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00012.safetensors'
INFO:hf-to-gguf:token_embd.weight,                 torch.bfloat16 --> BF16, shape = {4608, 256000}
INFO:hf-to-gguf:blk.0.attn_norm.w

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### We removed it in GGUF's chat template for you.


Unsloth: Conversion completed! Output location: ./gemma-2-27b/unsloth.Q5_K_M.gguf


Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/u54VK8m8tk) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Zephyr DPO 2x faster [free Colab](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing)
2. Llama 7b 2x faster [free Colab](https://colab.research.google.com/drive/1lBzz5KeZJKXjvivbYvmGarix9Ao6Wxe5?usp=sharing)
3. TinyLlama 4x faster full Alpaca 52K in 1 hour [free Colab](https://colab.research.google.com/drive/1AZghoNBQaMDgWJpi4RbffGM1h6raLUj9?usp=sharing)
4. CodeLlama 34b 2x faster [A100 on Colab](https://colab.research.google.com/drive/1y7A0AxE3y8gdj4AVkl2aZX47Xu3P1wJT?usp=sharing)
5. Mistral 7b [free Kaggle version](https://www.kaggle.com/code/danielhanchen/kaggle-mistral-7b-unsloth-notebook)
6. We also did a [blog](https://huggingface.co/blog/unsloth-trl) with 🤗 HuggingFace, and we're in the TRL [docs](https://huggingface.co/docs/trl/main/en/sft_trainer#accelerate-fine-tuning-2x-using-unsloth)!
7. `ChatML` for ShareGPT datasets, [conversational notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)
8. Text completions like novel writing [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)
9. [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Support our work if you can! Thanks!
</div>